In [32]:
!pip install --upgrade fidap 
import pandas as pd
from fidap import fidap_client
import config

In [27]:
fidap = fidap_client(api_key=config.FIDAP_API_KEY)

# Demographic Data Questions

### Which highly populated zip code had the majority of its population black americans in 2019?

In [28]:
fidap.sql("""
select 
    black_population / total_population as black_density,
    black_population, 
    total_population,
    zip_code,
    year 
from fidap-301014.census.census 
where year=2019 and total_population != 0 
order by black_population desc, black_density desc 
limit 1;
""")

,black_density,black_population,total_population,zip_code,year
0,0.859803,86706,100844,11236,2019


### Which highly populated zip code had the majority of its population white americans in 2019?

In [29]:
fidap.sql("""
select
    white_population / total_population as white_density,
    white_population,
    total_population,
    zip_code,
    year 
from fidap-301014.census.census 
where year=2019 and total_population != 0
order by white_population desc, white_density desc 
limit 1;
""")

,white_density,white_population,total_population,zip_code,year
0,0.870032,97113,111620,79936,2019


### What zip code has had the highest household income by median age?

In [31]:
fidap.sql("""
select
median_age, 
median_income, 
zip_code 
from fidap-301014.census.census 
where year=2019 and median_income is not null 
order by median_income desc, median_age asc 
limit 1;
""")

,median_age,median_income,zip_code
0,64.9,250000,82219


### Which zip code with a population of more than 50,000 recorded the highest population growth in any year?

In [51]:
fidap.sql("""
select zip_code, 
    total_population,
    LAG(total_population) over (partition by zip_code order by year) as previous_pop,
    IFNULL(total_population - (LAG(total_population) over (partition by zip_code order by year)), 0) as diff_to_prev,
    IFNULL(total_population - (LAG(total_population) over (partition by zip_code order by year)), 0) / total_population * 100 as percent_change,
    year 
from fidap-301014.census.census 
where total_population > 50000 
order by percent_change desc 
limit 1;
""")

,zip_code,total_population,previous_pop,diff_to_prev,percent_change,year
0,77407,60000,51142,8858,14.763333,2019


In [17]:
!pip show fidap

Name: fidap
Version: 0.0.9
Summary: Get financial data easily.
Home-page: https://github.com/fidapco/fidap-python
Author: Ashish Singal
Author-email: ashish.singal1@gmail.com
License: MIT
Location: /opt/conda/lib/python3.7/site-packages
Requires: requests, pandas, numpy, python-dotenv
Required-by: 
